Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All) to avoid typical problems with Jupyter notebooks. **Unfortunately, this does not work with Chrome right now, you will also need to reload the tab in Chrome afterwards**.

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE". Please put your name here:

In [1]:
NAME = "AVISHA ANILKUMAR BHIRYANI"

---

# Latent Dirichlet Allocation

Now we will use latent dirichlet allocation

**Important notice:** the "Validate" function might timeout at 30 or 60 seconds.
We intend to have the actual autograding later run with higher tolerance.

In [2]:
### Load the input data - do not modify
import json, gzip, numpy as np
raw = json.load(gzip.open("/data/simpsonswiki.json.gz", "rt", encoding="utf-8"))
titles, texts, classes = [x["title"] for x in raw], [x["text"] for x in raw], [x["c"] for x in raw]

In [3]:
### This cell reduces the data set size for the autograder tests - do not modify

In [4]:
### Vectorize the text - do not modify
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(stop_words="english", min_df=5)
counts = cvect.fit_transform(texts)
vocabulary = cvect.get_feature_names_out()

In [9]:
vocabulary

array(['00', '000', '01', ..., 'zzyzwicz', 'zörker', 'üter'], dtype=object)

## Explore your result

Explore the result: write a function to determine the most important words for each factor, and the most relevant documents.

**COPY your code from the first file here** (one of the rare cases, where copying is okay)

In [5]:
def most_important(vocabulary, factor, k=10):
    """Most important words for each factor"""
    # YOUR CODE HERE
    dict_words = {}
    i = 0
    for key in vocabulary:
        dict_words[key] = factor[i]
        i = i+1
    sorted_words = dict(sorted(dict_words.items(), key=lambda item: item[1], reverse=True))
    arr = [each for each in list(sorted_words)[:k]]
    return arr

def most_relevant(assignment, k=5):
    """Most relevant documents for each factor (return document indexes)"""
    # YOUR CODE HERE
    arr = []
    dict_docs = {}
    for i in range(0,len(assignment)):
        dict_docs[i] = assignment[i]
        i = i+1
    sorted_docs = dict(sorted(dict_docs.items(), key=lambda item: item[1], reverse=True))
    arr= [each for each in list(sorted_docs)[:k]]
    return arr

def explain(vocabulary, titles, classes, factors, assignment, weights=None):
    """Print an explanation for each factor.
       If weights is None, use the relative share of the assignment weights.
       Print the ARI when assigning each document to its maximum only."""
    from sklearn.metrics import adjusted_rand_score
    # YOUR CODE HERE
    if weights is not None:
        total_factors = factors.shape[0]
        for i in range(0,total_factors):
            important_words = most_important(vocabulary, factors[i])
            important_docs = most_relevant(assignment[:i])
            print(i)
            print(important_docs)
            print(important_words)

## Latent Dirichlet Allocation

Apply LDA from the `sklearn` package onto the data. Return the topic word distributions as well as the document topic distributions.

In [45]:
### Use sklearn LDA here:
def lda(counts, k):
    """Latent Dirichlet Allocation. Return the factors and document assignment"""
    # YOUR CODE HERE
    from sklearn.decomposition import LatentDirichletAllocation
    clf = LatentDirichletAllocation(n_components=k) 
    assignment = clf.fit_transform(counts)
    factors = clf.components_ / clf.components_.sum(axis=1)[:, np.newaxis]
    return factors, assignment

In [46]:
### Automatic tests. You do not need to understand or modify this code.
_tmp = lda(counts[:100], 2)
assert len(_tmp) == 2, "Incomplete result"
assert _tmp[0].shape == (2, counts.shape[1]), "Factor shape is not correct."
assert _tmp[1].shape == (100, 2), "Assignment shape is not correct."
assert abs(_tmp[0].sum()-2)<1e-7, "Topic word matrix are not probabilities."
assert abs(_tmp[1].sum()-100)<1e-7, "Document topic matrix are not probabilities."
del _tmp

In [47]:
# Explore your result. These must be meaningful topics!
# but the most relevant documents found here are not as good
lda_factors, lda_assignment = lda(counts, 6)
explain(vocabulary, titles, classes, lda_factors, lda_assignment)

In [48]:
### This cell contains additional tests. You do not need to modify this cell.